<a href="https://colab.research.google.com/github/SM24-Industrial-Software-Dev/ML-forecasting-NOx-levels/blob/ES-19-MSA-Data-Update/MSA_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is the setup code

In [ ]:
# Imports and Installations
from google.colab import userdata
import ee
import os
import requests
import zipfile
import io
import geemap

!pip install pycrs

In [ ]:
# Authenticate GEE
credentials = ee.ServiceAccountCredentials("yeshiva-summer-2024-1@yu-summer-2024.iam.gserviceaccount.com", key_data=userdata.get('GCP_CREDENTIALS'))
ee.Initialize(credentials = credentials, project='yu-summer-2024', opt_url='https://earthengine-highvolume.googleapis.com')

### This is the code for a class used to obtain a list of all MSAs and their boundaries

In [ ]:
# The class definition
class MSA:
    def __init__(self):
        """
        Initializes an object representing a collection of Metropolitan Statistical Areas (MSAs)
        """
        self._msa_low_res = self._retrieve_msas()
        self._names = self._msa_low_res.aggregate_array('NAME').getInfo()
        self._pop = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Count') \
            .filter(ee.Filter.calendarRange(2020, 2020, 'year')) \
            .mean()

    @property
    def all_areas(self):
      """
      Returns a FeatureCollection of all Metropolitan Statistical Areas (MSAs)
      """
      return self._pop.clipToCollection(self._msa_low_res)

    @property
    def names(self) -> list[str]:
      """
      Returns a list of all the MSA names.
      """
      return self._names

    def get_areas(self, names: str | list[str]):
      """
      Filters a FeatureCollection of MSAs by the selected name(s).

      Args:
        names (str or list[str]): The name(s) to filter by.

      Returns:
      """
      if isinstance(names, str):
        names = [names]
      # Return only the MSA geometries, not the clipped population image
      return self._msa_low_res.filter(ee.Filter.inList('NAME', names))

    def get_areas_pop(self, names: str | list[str] = None, areas: ee.FeatureCollection = None) -> tuple[str, int] | list[tuple[str, int]]:
      if not names and not areas:
        raise ValueError("Either 'names' or 'areas' must be provided.")
      if names and areas and len(names) != len(areas.aggregate_array('NAME').getInfo()):
        raise ValueError("'names' and 'areas' do not match.")
      if not names:
        names = areas.aggregate_array('NAME').getInfo()
      if not areas:
        areas = []
        for name in names:
          areas.append(self.get_areas(names))
      else:
        feature_list = areas.toList(areas.size())
        areas = []
        for area in feature_list:
          areas.append(ee.Feature(area))

      if isinstance(names, str) or len(names) == 1:
        if isinstance(names, str):
          names = [names]
        population_image = self._pop.clip(areas)
        population_sum = population_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=areas.geometry(),
            scale=925,
            maxPixels=1e13
        ).get('population_count')
        return (names[0], int(population_sum.getInfo()))
      else:
        pop_sums = []
        for name, feature in zip(names, areas):
          population_image = self._pop.clip(feature)
          population_sum = population_image.reduceRegion(
              reducer=ee.Reducer.sum(),
              geometry=feature.geometry(),
              scale=925,
              maxPixels=1e13
          ).get('population_count')
          pop_sums.append((name, int(population_sum.getInfo())))
        return pop_sums


    # Only used so far to obtain the low resolution shapefile, at a resolution of 1:20,000,000
    def _retrieve_msas(self, resolution='20m'):
      """
      Retrieves a shapefile containing all Core-Based Statistical Areas (CBSAs)

      Args:
          resolution (str): The resolution of the shapefile (20m by default, can be 500k, 5m, or 20m)

      Returns:
          FeatureCollection: A FeatureCollection of Metropolitan Statistical Areas (MSAs)
      """
      year = 2023
      filename = f'cb_{year}_us_cbsa_{resolution}'
      if not os.path.exists(f'/content/{filename}.shp'):
        # Download the shapefile
        response = requests.get(f'https://www2.census.gov/geo/tiger/GENZ{year}/shp/{filename}.zip')
        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Failed to download shapefile. Status code: {response.status_code}")
        # Extract the shapefile
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            zip_ref.extractall()

      # Upload the shapefile to GEE by reading it with Latin-1 encoding, which is commonly used for shapefiles
      cbsas = geemap.shp_to_ee(f'{filename}.shp', encoding='latin1')

      # Then filter for Metropolitan Statistical Areas (MSAs), which are CBSAs with a population > 50k
      return cbsas.filter(ee.Filter.eq('LSAD', 'M1'))